<a href="https://colab.research.google.com/github/jrcarob/notebooks/blob/master/IBEX35_sector_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/laresbernardo/notebooks/blob/master/Comparing_US_Sectors_Stocks.ipynb

# devtools::install_github("laresbernardo/lares")
library(lares)
library(dplyr)
library(ggplot2)

In [ ]:
symbols <-c("^IBEX", "ANA.MC", "ACX.MC", "ACS.MC", "AENA.MC", "ALM.MC", "AMS.MC", "MTS.MC")
from <- "2020-02-19"

In [ ]:
spain_sectors <- stocks_hist(symbols, from = from) %>%
  group_by(Symbol) %>%
  arrange(desc(Date)) %>%
  mutate(
    norm_close = Value / last(Value),
    type = if_else(Symbol == "IBEX", "Market", "Sector"),
    sector = case_when(
      Symbol == "^IBEX" ~ "IBEX35",
      Symbol == "ANA.MC" ~ "Industrial",
      Symbol == "ACX.MC" ~ "Industrial",
      Symbol == "ACS.MC" ~ "Industrial",
      Symbol == "AENA.MC" ~ "Transport",
      Symbol == "ALM.MC" ~ "Health",
      Symbol == "AMS.MC" ~ "Services",
      Symbol == "MTS.MC" ~ "Industrial",
      TRUE ~ "Other")) %>% 
  ungroup()


In [ ]:
cols <- names(lares::lares_pal()$palette)

ord <- spain_sectors %>% filter(Date == max(Date)) %>% 
  arrange(desc(norm_close)) %>% 
  mutate(sector = autoline(sector, 1)) %>% .$sector

In [ ]:
spain_sectors %>%
  mutate(
    sector = autoline(sector, 1),
    sector = factor(sector, levels = unique(ord))) %>%
  ggplot(aes(x = Date, y = norm_close, group = Symbol, colour = type)) +
  geom_rect(aes(xmin = min(Date), xmax = max(Date), ymin = -Inf, ymax = Inf), 
            fill = if_else(spain_sectors$type == "Market", cols[3], NULL), colour = "white") +
  geom_hline(yintercept = 1, linetype = "dashed", colour = "grey80") +
  geom_line(key_glyph = "timeseries") +
  facet_wrap(.~sector) +
  scale_colour_manual(values = cols[c(1, 4)]) +
  scale_y_percent() +
  labs(
    title = "IBEX-35 Sector Impact of Covid-19",
    subtitle = glued("Relative to {from}"),
    x = NULL, y = NULL, colour = NULL) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  theme_lares(legend = "top")